In [27]:
import os
import pandas as pd
from PIL import Image
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torchmetrics import Accuracy
import pytorch_lightning as pl

# Set paths for data
CHICKEN_IMAGES_PATH = "data/chicken-images/data"
DUCK_IMAGES_PATH = "data/duck-images/data"

# Create annotations for datasets
def create_annotations_file(paths):
    datasets = {'train': [], 'val': [], 'test': []}
    for phase in ['train', 'val', 'test']:
        for path in paths:
            full_path = os.path.join(path, phase)
            image_names = [os.path.join(full_path, img_name) for img_name in os.listdir(full_path)[:100]]
            label = path.split("/")[-2]
            datasets[phase].extend([(img, label) for img in image_names])
    
    return {x: pd.DataFrame(datasets[x], columns=["path", "class"]) for x in datasets}

# Load datasets
datasets = create_annotations_file([CHICKEN_IMAGES_PATH, DUCK_IMAGES_PATH])
train, val, test = datasets['train'], datasets['val'], datasets['test']

# Dataset class
class ChickenOrDuck(Dataset):
    def __init__(self, annotations_file, transform=None):
        self.annotations_file = annotations_file
        self.transform = transform

    def __len__(self):
        return len(self.annotations_file)

    def __getitem__(self, idx):
        img_path, img_label = self.annotations_file.iloc[idx]
        image = Image.open(img_path).convert('RGB')
        label = 1 if img_label == 'chicken' else 0
        if self.transform:
            image = self.transform(image)
        return image, label

# Transformations for image processing
transformations = {
    'train': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Datasets and Dataloaders
train_dataset = ChickenOrDuck(train, transformations['train'])
val_dataset = ChickenOrDuck(val, transformations['val'])
test_dataset = ChickenOrDuck(test, transformations['val'])

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=20)

# Define a Lightning Module for training
class ImageClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        backbone = models.resnet18(pretrained=True)
        num_ftrs = backbone.fc.in_features
        backbone.fc = nn.Linear(num_ftrs, 2)
        self.model = backbone
        self.accuracy = Accuracy(task='multiclass', num_classes=2)
        
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = self.accuracy(y_hat.softmax(dim=-1), y)
        self.log('train_loss', loss)
        self.log('train_acc', acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = self.accuracy(y_hat.softmax(dim=-1), y)
        self.log('val_loss', loss)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.model.parameters(), lr=2e-3)

# Training the model
trainer = pl.Trainer(max_epochs=25)
model = ImageClassifier()
trainer.fit(model, train_loader, val_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs



  | Name     | Type               | Params
------------------------------------------------
0 | model    | ResNet             | 11.2 M
1 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size (MB)


Epoch 24: 100%|██████████| 10/10 [00:06<00:00,  1.64it/s, v_num=1, train_acc=1.000, val_acc=1.000]

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: 100%|██████████| 10/10 [00:06<00:00,  1.56it/s, v_num=1, train_acc=1.000, val_acc=1.000]
